In [10]:
pip install deep_translator

Note: you may need to restart the kernel to use updated packages.


### Подключение библиотек

In [11]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import json
from deep_translator import GoogleTranslator



In [ ]:
result_list = {'title': [], 'year': [], 'duration': [], 'description': [], 'genre': [], 'country': []}

In [13]:
url = 'https://www.imdb.com/chart/top/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'Accept-Language': 'ru-RU,ru;q=0.9'
}
page = requests.get(url, headers=headers)

In [14]:
page.status_code

200

In [15]:
soup = bs(page.text, 'html.parser')

In [16]:
page.text

'<!DOCTYPE html><html lang="en-US" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><script>if(typeof uet === \'function\'){ uet(\'bb\', \'LoadTitle\', {wb: 1}); }</script><script>window.addEventListener(\'load\', (event) => {\n        if (typeof window.csa !== \'undefined\' && typeof window.csa === \'function\') {\n            var csaLatencyPlugin = window.csa(\'Content\', {\n                element: {\n                    slotId: \'LoadTitle\',\n                    type: \'service-call\'\n                }\n            });\n            csaLatencyPlugin(\'mark\', \'clickToBodyBegin\', 1744711296168);\n        }\n    })</script><title>IMDb Top 250 Movies</title><meta name="description" content="As rated by regular IMDb voters." data-id="main"/><meta name="robots" content="max-image-preview:large"/><script type="application/ld+json">{"@type":"ItemList","itemListEl

In [17]:
print(soup)

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><script>if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }</script><script>window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1744711296168);
        }
    })</script><title>IMDb Top 250 Movies</title><meta content="As rated by regular IMDb voters." data-id="main" name="description"/><meta content="max-image-preview:large" name="robots"/><script type="application/ld+json">{"@type":"ItemList","itemListElement":[{"@type":"ListItem","ite

In [18]:
# Для подгрузки вчех фильмов необходимо выполнить скрипт
script_tag = soup.find('script', {'id': '__NEXT_DATA__', 'type': 'application/json'})
data = json.loads(script_tag.string)

titles = []
for item in data['props']['pageProps']['pageData']['chartTitles']['edges']:
    title = item ['node']
    titles.append(title)

print(f"Найдено фильмов: {len(titles)}")


# Парсер заходит на страницу каждого фильма и получает необходимые поля
for i in titles:
    page = requests.get(f"https://www.imdb.com/title/{i['id']}/?ref_=chttp_t_1", headers=headers)
    soup = bs(page.text, 'html.parser')
    name_film = soup.find('span', class_='hero__primary-text')

    ul_year = soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt')
    year = ul_year.find('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color')
    duration = ul_year.findAll('li', class_='ipc-inline-list__item')
    describe = soup.find('span', class_="sc-42125d72-2 mmImo")
    script = soup.find('script', type='application/ld+json')
    genre = ""
    if script:
        data = json.loads(script.string)
        genre = data['genre']
    
    country = soup.find('div', {'data-testid' : "title-details-section"}).find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    
    describe_translate = GoogleTranslator(source='en', target='ru').translate(describe.text)
    genre_translate = GoogleTranslator(source='en', target='ru').translate(", ".join(genre))


    result_list['title'].append(name_film.text)

    result_list['year'].append(year.text)
    result_list['duration'].append(duration[-1].text)
    result_list['description'].append(describe_translate)
    result_list['genre'].append(genre_translate)
    result_list['country'].append(country[1].text)
    # print(f'{name_film.text} {year.text} {duration[-1].text}\n{describe_translate}\n{genre_translate}\n{country[1].text}')
    print(genre_translate)
    print(describe_translate)



Найдено фильмов: 250
Драма
Банкир, осужденный за уксорицид, образует дружбу за четверть века с закаленным осужденным, сохраняя при этом его невиновность и пытаясь оставаться надежды на простое сострадание.
Преступление, драма
Старение патриарха организованной криминальной династии передает контроль над его тайной империей его неохотному сыну.
Действие, преступность, драма
Когда угроза, известная как Джокер, наносит ущерб и хаос народах Готэма, Бэтмен, Джеймс Гордон и Харви Дент должны работать вместе, чтобы положить конец безумию.
Преступление, драма
Ранняя жизнь и карьера Вито Корлеоне в 1920 -х годах изображаются, в то время как его сын Майкл расширяет и подтягивает свою власть в отношении синдиката по семейной преступности.
Преступление, драма
Присяжные в суде по делу об убийстве в Нью -Йорке разочарованы одним членом, чей скептически осторожность заставляет их более тщательно рассмотреть доказательства, прежде чем принять поспешное приговор.
Приключения, драма, фантазия
Гандальф и 

In [23]:
result_list.pop('original_title', None)

[]

In [35]:
df = pd.DataFrame(result_list)
df.head(10)

,title,year,duration,description,genre,country
0,Побег из Шоушенка,1994,2h 22m,"Банкир, осужденный за уксорицид, образует друж...",Драма,United States
1,Крестный отец,1972,2h 55m,Старение патриарха организованной криминальной...,"Преступление, драма",United States
2,Тёмный рыцарь,2008,2h 32m,"Когда угроза, известная как Джокер, наносит ущ...","Действие, преступность, драма",United States
3,Крестный отец 2,1974,3h 22m,Ранняя жизнь и карьера Вито Корлеоне в 1920 -х...,"Преступление, драма",United States
4,12 разгневанных мужчин,1957,1h 36m,Присяжные в суде по делу об убийстве в Нью -Йо...,"Преступление, драма",United States
5,Властелин колец: Возвращение короля,2003,3h 21m,Гандальф и Арагорн ведут мир людей против арми...,"Приключения, драма, фантазия",New Zealand
6,Список Шиндлера,1993,3h 15m,В оккупированной немецкой Поль году во время В...,"Биография, драма, история",United States
7,Криминальное чтиво,1994,2h 34m,"Жизнь двух наемников толпы, боксера, гангстера...","Преступление, драма",United States
8,Властелин колец: Братство кольца,2001,2h 58m,Крутый хоббит из Шира и восемь компаньонов отп...,"Приключения, драма, фантазия",New Zealand
9,"Хороший, плохой, злой",1966,2h 41m,Мошенничество на щедрости присоединяется к дву...,"Приключение, западный",Italy


In [36]:
# Сохранил датасет в файл
df.to_csv('films_dataset.csv')

In [37]:

df = pd.read_csv('films_dataset.csv')

In [38]:
df.head()

,Unnamed: 0,title,year,duration,description,genre,country
0,0,Побег из Шоушенка,1994,2h 22m,"Банкир, осужденный за уксорицид, образует друж...",Драма,United States
1,1,Крестный отец,1972,2h 55m,Старение патриарха организованной криминальной...,"Преступление, драма",United States
2,2,Тёмный рыцарь,2008,2h 32m,"Когда угроза, известная как Джокер, наносит ущ...","Действие, преступность, драма",United States
3,3,Крестный отец 2,1974,3h 22m,Ранняя жизнь и карьера Вито Корлеоне в 1920 -х...,"Преступление, драма",United States
4,4,12 разгневанных мужчин,1957,1h 36m,Присяжные в суде по делу об убийстве в Нью -Йо...,"Преступление, драма",United States


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   250 non-null    int64 
 1   title        250 non-null    object
 2   year         250 non-null    int64 
 3   duration     250 non-null    object
 4   description  250 non-null    object
 5   genre        250 non-null    object
 6   country      250 non-null    object
dtypes: int64(2), object(5)
memory usage: 13.8+ KB
